In [4]:
import os
import mlflow
import numpy as np
import pandas as pd

from math import sqrt
from joblib import delayed
from joblib import Parallel

from warnings import catch_warnings
from warnings import filterwarnings
from multiprocessing import cpu_count
from sklearn.metrics import mean_squared_error
from statsmodels.tsa.statespace.sarimax import SARIMAX

import matplotlib.pyplot as plt
import seaborn as sns; sns.set()

from pathlib import Path
from functools import reduce
from datetime import datetime

from prophet import Prophet

Importing plotly failed. Interactive plots will not work.


# Data

In [5]:
df = pd.read_csv('./../final_dfs/for_modeling/df_final_date_wide_2007.csv', converters={'ags2': str, 'ags5': str} )
df.set_index('ags5', drop=True, inplace=True)
df = df.drop("Unnamed: 0", axis=1)
df['date'] = pd.to_datetime(df['date'], format = '%Y-%m-%d')
df = df[(df['date'] >= '2010-01-01') & (df['date'] < '2020-01-01')]
df = df.rename(columns={'date': 'ds', 'unemployment_rate': 'y'})

# 401 Model for each kreis

In [9]:
# one-step forecast
def last_obs(history):
    res = history.iloc[-1]["y"]
    return [res] * 12

In [10]:
config = ['linear', 15.0, True, 'additive']
last_obs(df)

[6.5, 6.5, 6.5, 6.5, 6.5, 6.5, 6.5, 6.5, 6.5, 6.5, 6.5, 6.5]

In [11]:
# root mean squared error or rmse
def measure_rmse(actual, predicted):
    return sqrt(mean_squared_error(actual, predicted))

In [12]:
# split a univariate dataset into train/test sets
def train_test_split(data, n_test):
    return data[:-n_test], data[-n_test:]

In [57]:
(train_test_split(df, 12)[1] )

,ags2,ds,number_of_company_deletions,number_of_company_liquidations,number_of_start_ups,number_of_companies_administration,number_of_companies_agriculture,number_of_companies_arts_entertainment,number_of_companies_communication,number_of_companies_construction,...,realized_short_time_work_companies,realized_short_time_work_people,underemployment_without_short_time _work,unemployment_benefit_entitled,unemployment_benefit_recipients,y,registerd_jobs,unemployed,employees_social_security_at_work,employees_social_security_at_residence
ags5,,,,,,,,,,,,,,,,,,,,,
16077,16,2019-01-01,7.0,4.0,5.0,93.0,106.0,97.0,38.0,486.0,...,NaN,NaN,4836.0,7762.0,1012.0,8.0,997,3675,27701,33710
16077,16,2019-02-01,4.0,1.0,4.0,-99.0,-99.0,-99.0,-99.0,-99.0,...,NaN,NaN,4842.0,7711.0,1005.0,8.0,1014,3673,27702,33723
16077,16,2019-03-01,2.0,1.0,2.0,101.0,84.0,77.0,34.0,425.0,...,NaN,NaN,4740.0,7685.0,927.0,7.6,1048,3489,27818,33784
16077,16,2019-04-01,3.0,0.0,1.0,111.0,95.0,111.0,39.0,423.0,...,NaN,NaN,4615.0,7602.0,822.0,7.2,1107,3319,27992,33885
16077,16,2019-05-01,7.0,0.0,2.0,112.0,101.0,123.0,44.0,482.0,...,NaN,NaN,4575.0,7495.0,763.0,7.1,1059,3214,27960,33890
16077,16,2019-06-01,5.0,0.0,1.0,111.0,95.0,120.0,42.0,475.0,...,NaN,NaN,4497.0,7387.0,767.0,7.0,1047,3140,27912,33883
16077,16,2019-07-01,1.0,3.0,4.0,111.0,96.0,120.0,42.0,475.0,...,NaN,NaN,4504.0,7294.0,821.0,7.0,1044,3146,27829,33779
16077,16,2019-08-01,3.0,3.0,2.0,111.0,96.0,121.0,42.0,477.0,...,NaN,NaN,4513.0,7188.0,842.0,7.0,990,3168,27980,33983
16077,16,2019-09-01,1.0,1.0,3.0,111.0,93.0,116.0,38.0,451.0,...,NaN,NaN,4390.0,7145.0,786.0,6.5,984,2929,28228,34292


In [18]:
# walk-forward validation for univariate data
def walk_forward_validation(data, n_test):
    # split dataset
    train, test = train_test_split(data, n_test)
    # fit model and make forecast for history
    yhat = last_obs(train)
    # estimate prediction error
    error = measure_rmse(test['y'], yhat)
    return error

In [19]:
def run_model(data, n_test): 
    with mlflow.start_run():
        ags5 = list(set(df.index.values.tolist()))
        rmse = []
        for i in ags5:
            df_ags5 = df.filter(regex=i, axis=0)
            current_rmse = walk_forward_validation(df_ags5, n_test)
            print(current_rmse)
            rmse.append(current_rmse)

        # Log params
        params = {"growth":"NONE"}

        mlflow.log_params(params)

        # Log metrics
        mlflow.log_metric('rmse', np.mean(rmse))

In [20]:
run_model(df, 12)

1.0579067381705562
0.23804761428476162
0.25819888974716093
0.28431203515386616
0.6988085097745541
0.39475730941090037
0.24664414311581218
0.9733961166965893
0.3851406669430447
0.2986078811194822
0.25166114784235843
0.2020725942163689
0.19364916731037088
0.7427426651719065
0.5041494487418058
0.27080128015453225
0.17078251276599296
0.35590260840104343
0.3818813079129868
0.2843120351538662
1.1644025649805712
0.2217355782608345
0.7153087911291643
0.19364916731037082
0.5283622494715786
0.36742346141747667
0.27080128015453203
0.7135591542869212
1.5858751527153705
0.1414213562373093
0.4453463071962463
0.37969285832981753
0.1848422751068231
0.22912878474779197
0.21984843263788176
0.45916591046519695
0.30138568866708537
0.2614064523559687
0.3535533905932739
0.39999999999999997
0.25658007197234417
0.32403703492039254
0.19790570145063202
0.3068658773253662
0.3851406669430449
0.14999999999999994
0.16329931618554505
0.17320508075688767
0.9478044805408623
0.43301270189221897
0.12583057392117908
0.33